In [ ]:
from sklearn.metrics import mean_absolute_error,r2_score, mean_squared_error
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.preprocessing import MinMaxScaler
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import torch.optim as optim
import os
import random
import gc


def set_deterministic(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_deterministic(42)

## 0. Dataset and DataLoader

In [ ]:
class WindPowerDataset(Dataset):
    def __init__(self, csv_file, wind_power_scaler=None, weather_scaler=None, save_scalers=False):
        self.data = pd.read_csv(csv_file)
        
        # 采样每5行取一行
        self.data = self.data.iloc[::5, :].reset_index(drop=True)
        
        # 计算原始数据的标准差
        self.original_wind_power_std = self.data.iloc[:, 2].std()
        self.original_weather_std = self.data.iloc[:, 4:12].std()
        
        # 初始化归一化器
        if wind_power_scaler is None or weather_scaler is None:
            self.wind_power_scaler = MinMaxScaler()
            self.weather_scaler = MinMaxScaler()
            
            # 对风功率数据进行归一化
            self.data.iloc[:, 2] = self.wind_power_scaler.fit_transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()
            
            # 对天气数据进行归一化
            self.data.iloc[:, 4:12] = self.weather_scaler.fit_transform(self.data.iloc[:, 4:12])
            
            if save_scalers:
                with open('wind_power_scaler.pkl', 'wb') as f:
                    pickle.dump(self.wind_power_scaler, f)
                with open('weather_scaler.pkl', 'wb') as f:
                    pickle.dump(self.weather_scaler, f)
        else:
            self.wind_power_scaler = wind_power_scaler
            self.weather_scaler = weather_scaler
            self.data.iloc[:, 2] = self.wind_power_scaler.transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()
            self.data.iloc[:, 4:12] = self.weather_scaler.transform(self.data.iloc[:, 4:12])
    
    def __len__(self):
        return len(self.data) - 312  # 288 (1440/5) + 24 (120/5)
    
    def __getitem__(self, idx):
        wind_power_history = self.data.iloc[idx:idx + 288, 2].values.astype(float)
        future_weather = self.data.iloc[idx + 288:idx + 312, 4:12].values.astype(float)
        future_wind_power = self.data.iloc[idx + 312, 2]
        return torch.tensor(wind_power_history, dtype=torch.float32), \
               torch.tensor(future_weather, dtype=torch.float32), \
               torch.tensor(future_wind_power, dtype=torch.float32)
    
    def get_original_stds(self):
        return {
            'original_wind_power_std': self.original_wind_power_std,
            'original_weather_std': self.original_weather_std.to_dict()
        }

In [ ]:
dataset = WindPowerDataset('CAISO_zone_1_.csv', save_scalers=True)
weather_stds = dataset.get_original_stds()
total_size = len(dataset)
train_size = int(0.8 * total_size)  
test_size = total_size - train_size  
train_idx = list(range(train_size))
test_idx = list(range(train_size, total_size))

train_dataset = Subset(dataset, train_idx)
test_dataset = Subset(dataset, test_idx)

batch_size = 32  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)   

wind_power_scaler=dataset.wind_power_scaler

In [ ]:
def create_train_loader(csv_file, batch_size=32):
    dataset = WindPowerDataset(csv_file, save_scalers=False)  
    total_size = len(dataset)
    train_size = int(0.8 * total_size)
    train_idx = list(range(train_size))
    train_subset = torch.utils.data.Subset(dataset, train_idx)
    train_loader = torch.utils.data.DataLoader(
        train_subset, batch_size=batch_size, shuffle=False, num_workers=0
    )
    return train_loader, dataset  

## 1. UP

In [ ]:
def optimize_universal_perturbation(model, data_loader, epsilon=0.01, device='cpu', 
                                   num_epochs=100, lr=0.01, lambda_reg=0.0001, seed=42):
    
    generator = torch.Generator(device=device)
    generator.manual_seed(seed)

    model.train()  

    for param in model.parameters():
        param.requires_grad_(False)

    perturbation_shape = (24, 8)
    universal_perturbation = torch.rand(
        perturbation_shape, generator=generator, device=device
    ) * 2 * epsilon - epsilon
    universal_perturbation.requires_grad_(True)

    optimizer = torch.optim.Adam([universal_perturbation], lr=lr)

    best_perturbation = universal_perturbation.clone().detach()
    best_loss = float('inf')

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for wind_history, weather_future, future_wind_power in data_loader:
            wind_history = wind_history.to(device)
            weather_future = weather_future.to(device)
            future_wind_power = future_wind_power.to(device)

            adv_weather = weather_future + universal_perturbation.unsqueeze(0)
            adv_weather = torch.clamp(adv_weather, 0.0, 1.0)

            output_adv = model(wind_history, adv_weather).squeeze()

            loss = -torch.mean(output_adv - future_wind_power)

            optimizer.zero_grad()
            loss.backward()       
            optimizer.step()

            with torch.no_grad():
                universal_perturbation.clamp_(-epsilon, epsilon)

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(data_loader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            best_perturbation = universal_perturbation.clone().detach()

        if (epoch + 1) % 5 == 0:
            print(f'Epoch {epoch+1}, Avg Loss: {avg_loss:.6f}')

    return best_perturbation

### UP-LSTM

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)
        return output
def load_model(model_path, device='cpu'):
    model =WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model
model_path = 'wind_caiso_lstm_sigmoid.pth' 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_model(model_path, device)

In [ ]:
print("Running epsilon=0.10...")
uap_list010 = []
for i in range(20):
    seed = 1000 + i

    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.10, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list010.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")

    del train_loader, dataset
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_lstm_epsilon_010.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list010,
        'seeds': list(range(1000, 1020)),
        'epsilon': 0.10
    }, f)

del uap_list010
gc.collect()
print("Saved and cleared epsilon=0.10\n")

In [ ]:
print("Running epsilon=0.05...")
uap_list005 = []
for i in range(20):
    seed = 2000 + i

    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.05, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list005.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")

    del train_loader, dataset
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_lstm_epsilon_005.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list005,
        'seeds': list(range(2000, 2020)),
        'epsilon': 0.05
    }, f)

del uap_list005
gc.collect()
print("Saved and cleared epsilon=0.05\n")

In [ ]:
print("Running epsilon=0.03...")
uap_list003 = []
for i in range(20):
    seed = 3000 + i
    
    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.03, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list003.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")
    
    del train_loader, dataset  
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_lstm_epsilon_003.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list003,
        'seeds': list(range(3000, 3020)),
        'epsilon': 0.03
    }, f)

del uap_list003
gc.collect()
print("Saved and cleared epsilon=0.03\n")

### UP-Transformer

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self, d_model=50):
        super(WindPowerPredictor, self).__init__()
        self.d_model = d_model
        self.embedding_wind = nn.Linear(1, d_model)
        self.embedding_weather = nn.Linear(8, d_model)
        self.transformer_wind = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.transformer_weather = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.fc = nn.Linear(d_model * 2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, wind_history, weather_future):
        wind_history = self.embedding_wind(wind_history.unsqueeze(-1))  # (batch_size, seq_len, d_model)
        wind_history = wind_history.permute(1, 0, 2)  # (seq_len, batch_size, d_model)
        weather_future = self.embedding_weather(weather_future)  # (batch_size, seq_len, d_model)
        weather_future = weather_future.permute(1, 0, 2)  # (seq_len, batch_size, d_model)

        transformer_output_wind = self.transformer_wind(wind_history, wind_history)
        transformer_output_weather = self.transformer_weather(weather_future, weather_future)
        
        combined = torch.cat((transformer_output_wind[-1, :, :], transformer_output_weather[-1, :, :]), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)  
        return output

In [ ]:
def load_model(model_path, device='cpu'):
    model = WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = 'wind_caiso_transformer_sigmoid.pth'
model = load_model(model_path, device)

In [ ]:
print("Running epsilon=0.10...")
uap_list010 = []
for i in range(20):
    seed = 1000 + i

    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.10, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list010.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")

    del train_loader, dataset
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_transformer_epsilon_010.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list010,
        'seeds': list(range(1000, 1020)),
        'epsilon': 0.10
    }, f)

del uap_list010
gc.collect()
print("Saved and cleared epsilon=0.10\n")

print("Running epsilon=0.05...")
uap_list005 = []
for i in range(20):
    seed = 2000 + i

    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.05, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list005.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")

    del train_loader, dataset
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_transformer_epsilon_005.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list005,
        'seeds': list(range(2000, 2020)),
        'epsilon': 0.05
    }, f)

del uap_list005
gc.collect()
print("Saved and cleared epsilon=0.05\n")

print("Running epsilon=0.03...")
uap_list003 = []
for i in range(20):
    seed = 3000 + i
    
    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.03, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list003.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")
    
    del train_loader, dataset  
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_transformer_epsilon_003.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list003,
        'seeds': list(range(3000, 3020)),
        'epsilon': 0.03
    }, f)

del uap_list003
gc.collect()
print("Saved and cleared epsilon=0.03\n")

### UP-GRU

In [ ]:
class WindPowerPredictor(nn.Module):  
    def __init__(self):  
        super(WindPowerPredictor, self).__init__()  
        self.gru_wind = nn.GRU(input_size=1, hidden_size=50, num_layers=2, batch_first=True)  
        self.gru_weather = nn.GRU(input_size=8, hidden_size=50, num_layers=2, batch_first=True)  
        self.fc = nn.Linear(100, 1)  
        self.sigmoid = nn.Sigmoid()
      
    def forward(self, wind_history, weather_future):  
        wind_history = wind_history.unsqueeze(-1)  
        _, hn_wind = self.gru_wind(wind_history)  
        _, hn_weather = self.gru_weather(weather_future)  
        hn_wind = hn_wind[-1, :, :]  
        hn_weather = hn_weather[-1, :, :]  
        combined = torch.cat((hn_wind, hn_weather), dim=1)  
        output = self.fc(combined)  
        output = self.sigmoid(output)
        return output  
model_path = 'wind_gru_caiso_sigmoid.pth'
model = load_model(model_path, device)

In [ ]:
print("Running epsilon=0.10...")
uap_list010 = []
for i in range(20):
    seed = 1000 + i

    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.10, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list010.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")

    del train_loader, dataset
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_gru_epsilon_010.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list010,
        'seeds': list(range(1000, 1020)),
        'epsilon': 0.10
    }, f)

del uap_list010
gc.collect()
print("Saved and cleared epsilon=0.10\n")

print("Running epsilon=0.05...")
uap_list005 = []
for i in range(20):
    seed = 2000 + i

    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.05, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list005.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")

    del train_loader, dataset
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_gru_epsilon_005.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list005,
        'seeds': list(range(2000, 2020)),
        'epsilon': 0.05
    }, f)

del uap_list005
gc.collect()
print("Saved and cleared epsilon=0.05\n")

print("Running epsilon=0.03...")
uap_list003 = []
for i in range(20):
    seed = 3000 + i
    
    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.03, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list003.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")
    
    del train_loader, dataset  
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_gru_epsilon_003.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list003,
        'seeds': list(range(3000, 3020)),
        'epsilon': 0.03
    }, f)

del uap_list003
gc.collect()
print("Saved and cleared epsilon=0.03\n")

### UP-TCN

In [ ]:
model_path = 'wind_tcn_caiso_sigmoid.pth'
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=(kernel_size-1) * dilation_size, dilation=dilation_size),
                       nn.ReLU(),
                       nn.Dropout(dropout)]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class WindPowerPredictorTCN(nn.Module):
    def __init__(self):
        super(WindPowerPredictorTCN, self).__init__()
        self.tcn_wind = TemporalConvNet(num_inputs=1, num_channels=[50]*3, kernel_size=3, dropout=0.2)
        self.tcn_weather = TemporalConvNet(num_inputs=8, num_channels=[50]*3, kernel_size=3, dropout=0.2)
        self.fc = nn.Linear(50 * 2, 1)  # Combined output size of TCNs
        self.sigmoid = nn.Sigmoid()

    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(1)  # (batch_size, 1, seq_len)
        tcn_output_wind = self.tcn_wind(wind_history).transpose(1, 2)[:, -1, :]
        
        weather_future = weather_future.transpose(1, 2)  # (batch_size, 8, seq_len)
        tcn_output_weather = self.tcn_weather(weather_future).transpose(1, 2)[:, -1, :]
        
        combined = torch.cat((tcn_output_wind, tcn_output_weather), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)
        return output
def load_model(model_path, device='cpu'):
    model = WindPowerPredictorTCN().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model
model = load_model(model_path, device)

In [ ]:
print("Running epsilon=0.10...")
uap_list010 = []
for i in range(20):
    seed = 1000 + i

    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.10, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list010.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")

    del train_loader, dataset
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_tcn_epsilon_010.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list010,
        'seeds': list(range(1000, 1020)),
        'epsilon': 0.10
    }, f)

del uap_list010
gc.collect()
print("Saved and cleared epsilon=0.10\n")

print("Running epsilon=0.05...")
uap_list005 = []
for i in range(20):
    seed = 2000 + i

    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.05, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list005.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")

    del train_loader, dataset
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_tcn_epsilon_005.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list005,
        'seeds': list(range(2000, 2020)),
        'epsilon': 0.05
    }, f)

del uap_list005
gc.collect()
print("Saved and cleared epsilon=0.05\n")

print("Running epsilon=0.03...")
uap_list003 = []
for i in range(20):
    seed = 3000 + i
    
    train_loader, dataset = create_train_loader('CAISO_zone_1_.csv', batch_size=32)
    
    uap = optimize_universal_perturbation(
        model, train_loader, epsilon=0.03, device=device,
        num_epochs=20, lr=0.00001, lambda_reg=0, seed=seed
    )
    uap_list003.append(uap.cpu().numpy())
    print(f"  Done {i+1}/20")
    
    del train_loader, dataset  
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open('uap_results_tcn_epsilon_003.pkl', 'wb') as f:
    pickle.dump({
        'uap_list': uap_list003,
        'seeds': list(range(3000, 3020)),
        'epsilon': 0.03
    }, f)

del uap_list003
gc.collect()
print("Saved and cleared epsilon=0.03\n")

## RUP

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)
        return output
def load_model(model_path, device='cpu'):
    model =WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model
model_path1 = 'wind_caiso_lstm_sigmoid.pth' 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model1 = load_model(model_path1, device)

class WindPowerPredictor(nn.Module):
    def __init__(self, d_model=50):
        super(WindPowerPredictor, self).__init__()
        self.d_model = d_model
        self.embedding_wind = nn.Linear(1, d_model)
        self.embedding_weather = nn.Linear(8, d_model)
        self.transformer_wind = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.transformer_weather = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.fc = nn.Linear(d_model * 2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, wind_history, weather_future):
        wind_history = self.embedding_wind(wind_history.unsqueeze(-1))  # (batch_size, seq_len, d_model)
        wind_history = wind_history.permute(1, 0, 2)  # (seq_len, batch_size, d_model)
        weather_future = self.embedding_weather(weather_future)  # (batch_size, seq_len, d_model)
        weather_future = weather_future.permute(1, 0, 2)  # (seq_len, batch_size, d_model)

        transformer_output_wind = self.transformer_wind(wind_history, wind_history)
        transformer_output_weather = self.transformer_weather(weather_future, weather_future)
        
        combined = torch.cat((transformer_output_wind[-1, :, :], transformer_output_weather[-1, :, :]), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)  
        return output
    
def load_model(model_path, device='cpu'):
    model = WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model
model_path2 = 'wind_caiso_transformer_sigmoid.pth'
model2 = load_model(model_path2, device)

class WindPowerPredictor(nn.Module):  
    def __init__(self):  
        super(WindPowerPredictor, self).__init__()  
        self.gru_wind = nn.GRU(input_size=1, hidden_size=50, num_layers=2, batch_first=True)  
        self.gru_weather = nn.GRU(input_size=8, hidden_size=50, num_layers=2, batch_first=True)  
        self.fc = nn.Linear(100, 1)  
        self.sigmoid = nn.Sigmoid()
      
    def forward(self, wind_history, weather_future):  
        wind_history = wind_history.unsqueeze(-1)  
        _, hn_wind = self.gru_wind(wind_history)  
        _, hn_weather = self.gru_weather(weather_future)  
        hn_wind = hn_wind[-1, :, :]  
        hn_weather = hn_weather[-1, :, :]  
        combined = torch.cat((hn_wind, hn_weather), dim=1)  
        output = self.fc(combined)  
        output = self.sigmoid(output)
        return output  
model_path3 = 'wind_gru_caiso_sigmoid.pth'
model3 = load_model(model_path3, device)

model_path4 = 'wind_tcn_caiso_sigmoid.pth'
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=(kernel_size-1) * dilation_size, dilation=dilation_size),
                       nn.ReLU(),
                       nn.Dropout(dropout)]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class WindPowerPredictorTCN(nn.Module):
    def __init__(self):
        super(WindPowerPredictorTCN, self).__init__()
        self.tcn_wind = TemporalConvNet(num_inputs=1, num_channels=[50]*3, kernel_size=3, dropout=0.2)
        self.tcn_weather = TemporalConvNet(num_inputs=8, num_channels=[50]*3, kernel_size=3, dropout=0.2)
        self.fc = nn.Linear(50 * 2, 1)  # Combined output size of TCNs
        self.sigmoid = nn.Sigmoid()

    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(1)  # (batch_size, 1, seq_len)
        tcn_output_wind = self.tcn_wind(wind_history).transpose(1, 2)[:, -1, :]
        
        weather_future = weather_future.transpose(1, 2)  # (batch_size, 8, seq_len)
        tcn_output_weather = self.tcn_weather(weather_future).transpose(1, 2)[:, -1, :]
        
        combined = torch.cat((tcn_output_wind, tcn_output_weather), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)
        return output
def load_model(model_path, device='cpu'):
    model = WindPowerPredictorTCN().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model
model4 = load_model(model_path4, device)

In [ ]:
def adversarial_attack_inde(uap_loaded, model, data_loader,  device='cpu'):
    model.eval()  
    all_preds = []
    all_adversarial_preds = []
    all_targets = []
    all_original_weather = []
    all_adversarial_weather = []
    
    for wind_history, weather_future, future_wind_power in data_loader:
        wind_history = wind_history.to(device)
        weather_future = weather_future.to(device)
        future_wind_power = future_wind_power.to(device)

        original_output = model(wind_history, weather_future)

        adversarial_weather = weather_future + uap_loaded
        adversarial_weather = torch.clamp(adversarial_weather, 0, 1)  

        adversarial_output = model(wind_history, adversarial_weather.detach())  

        all_preds.append(original_output.detach().cpu().numpy())
        all_adversarial_preds.append(adversarial_output.detach().cpu().numpy())
        all_targets.append(future_wind_power.cpu().numpy())
        all_original_weather.append(weather_future.detach().cpu().numpy())
        all_adversarial_weather.append(adversarial_weather.detach().cpu().numpy())

    model.eval()  
    return all_preds, all_adversarial_preds, all_targets, all_original_weather, all_adversarial_weather

In [ ]:
proxy_models = [model1, model2, model3, model4]
model_names = ['tcn', 'lstm', 'gru', 'transformer']

def compute_uap_mae_on_all_proxies(uap_np, proxy_models, test_loader, scaler, device):

    uap_tensor = torch.tensor(uap_np, dtype=torch.float32).to(device)
    maes = []
    
    for model in proxy_models:
        model.eval()
        _, adv_preds, targets, _, _ = adversarial_attack_inde(
            uap_tensor, model, test_loader, device=device
        )
        
        adv_flat = np.concatenate(adv_preds).flatten()
        tgt_flat = np.concatenate(targets).flatten()

        tgt_inv = scaler.inverse_transform(tgt_flat.reshape(-1, 1)).squeeze()
        pred_inv = scaler.inverse_transform(adv_flat.reshape(-1, 1)).squeeze()
        
        mae = mean_absolute_error(tgt_inv, pred_inv)
        maes.append(mae)
    
    return np.mean(maes)  

epsilons = [('010', 0.10), ('005', 0.05), ('003', 0.03)]
all_mae_lists = {}  

for suffix, eps_val in epsilons:
    print(f"\n Processing epsilon = {eps_val} ({suffix})")
    
    all_uaps = []     
    mae_list = []      

    for name in model_names:
        pkl_path = f'uap_results_{name}_epsilon_{suffix}.pkl'
        print(f"  Loading {pkl_path}")
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            uap_list = data['uap_list']  

        for uap_np in uap_list:
            avg_mae = compute_uap_mae_on_all_proxies(
                uap_np, proxy_models, test_loader, wind_power_scaler, device
            )
            mae_list.append(avg_mae)
            all_uaps.append(uap_np)  

    all_mae_lists[suffix] = mae_list
    print(f" Completed epsilon={eps_val}: {len(mae_list)} MAEs computed.")

mae_list010 = all_mae_lists['010']  
mae_list005 = all_mae_lists['005']
mae_list003 = all_mae_lists['003']

print("\n All MAE lists ready!")

In [ ]:
from scipy.spatial.distance import cdist

model_names = ['tcn', 'lstm', 'gru', 'transformer']

def load_all_uaps_for_epsilon(suffix):
    all_uaps = []
    for name in model_names:
        pkl_path = f'uap_results_{name}_epsilon_{suffix}.pkl'
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            for uap in data['uap_list']:
                if isinstance(uap, torch.Tensor):
                    uap = uap.cpu().numpy()
                else:
                    uap = np.asarray(uap) 
                all_uaps.append(uap)

    return np.array(all_uaps)

def calculate_local_density(uaps, sigma=1.0):
    uaps_flat = uaps.reshape(uaps.shape[0], -1)  
    distances = cdist(uaps_flat, uaps_flat, 'euclidean')
    exp_distances = np.exp(-distances**2 / (2 * sigma**2))
    densities = np.sum(exp_distances, axis=1) - 1  
    return densities

def calculate_weighted_density(densities, deviations):
    weighted_products = densities * deviations   
    total_weighted_product = np.sum(weighted_products)
    if total_weighted_product == 0:
        return np.ones_like(weighted_products) / len(weighted_products)
    weighted_densities = weighted_products / total_weighted_product
    return weighted_densities

def combine_ups(uaps, weighted_densities):
    rup = np.sum([density * up for density, up in zip(weighted_densities, uaps)], axis=0)
    return rup

configurations = [
    ('010', mae_list010),
    ('005', mae_list005),
    ('003', mae_list003),
]

for suffix, mae_list in configurations:
    print(f"Processing RUP for epsilon={suffix}")

    uaps = load_all_uaps_for_epsilon(suffix)  
    deviations = np.array(mae_list)           

    assert uaps.shape[0] == len(deviations), "UAP count mismatch!"

    local_densities = calculate_local_density(uaps, sigma=0.02)
    weighted_densities = calculate_weighted_density(local_densities, deviations)
    rup_ensemble = combine_ups(uaps, weighted_densities)
    
    filename = f'rup_ensemble_{suffix}_caiso.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(rup_ensemble, f)
    
    print(f"Saved {filename} | RUP shape: {rup_ensemble.shape}")

## RUPW

In [ ]:
def load_all_uaps_numpy(suffix):
    all_uaps = []
    for name in model_names:
        path = f'uap_results_{name}_epsilon_{suffix}.pkl'
        with open(path, 'rb') as f:
            data = pickle.load(f)
            for uap in data['uap_list']:
                if isinstance(uap, torch.Tensor):
                    uap = uap.cpu().numpy()
                else:
                    uap = np.asarray(uap)
                all_uaps.append(uap)
    return np.array(all_uaps)  

for suffix in ['010', '005', '003']:
    print(f"Computing simple average RUPW for epsilon={suffix}")
    
    uaps = load_all_uaps_numpy(suffix)  # (80, 24, 8)
    rupw = np.mean(uaps, axis=0)        # (24, 8)
    
    filename = f'RUPW{suffix}_caiso.pkl'

    with open(filename, 'wb') as f:
        pickle.dump(rupw, f)
    
    print(f"Saved {filename} | Shape: {rupw.shape}")